# Introduction to NLP fundamentals in Tensorflow

NLP has the goal of deriving information out of natural langugae (could be sequences, text or speech).

Another common term for NLP problems is seuence to sequence problems (seq2seq).


## Check for GPU

In [ ]:
# !nvidia-smi -L

## Get helper functions

In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

# Import series of helper functions for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2021-11-16 06:35:04--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py.1’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2021-11-16 06:35:04 (69.1 MB/s) - ‘helper_functions.py.1’ saved [10246/10246]



## Get a text dataset

The dataset we're going to be using is Kaggle's introduction to NLP dataset (text samples of tweets labelled as disaster or not disaster).

Dataset: https://www.kaggle.com/c/nlp-getting-started/data

In [3]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

# unzip data
unzip_data("nlp_getting_started.zip")

--2021-11-16 06:35:43--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.8.128, 74.125.23.128, 64.233.189.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.8.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.005s  

2021-11-16 06:35:43 (115 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Visualizing a text dataset

To visualize our text samples, we first have to read them in. One way to do so would be to use python: https://realpython.com/read-write-files-python/

Another way to do this is to use pandas.

In [4]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
train_df["text"][10]

'Three people died from the heat wave so far'

In [6]:
# shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [7]:
train_df_shuffled["text"].iloc[3]

'Aftershock back to school kick off was great. I want to thank everyone for making it possible. What a great night.'

In [8]:
# what does the dataframe look like?
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [9]:
# how many examples of each class?
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [10]:
# how many total samples?
len(train_df), len(test_df)

(7613, 3263)

In [11]:
# Let's visualize some random training examples
import random

random_index = random.randint(0, len(train_df)-5) # create random indexes not higher than the total number of samples
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real disaster)" if target> 0 else "(not real disaster)")
  print(f"Text:{text}\n")
  print("----\n")

Target: 0 (not real disaster)
Text:@Sweet2Young I came in! Had he fucking moved his entire existence would've been obliterated.

----

Target: 0 (not real disaster)
Text:Businesses are deluged with invoices. Make yours stand ogt with colomr or shape and it's likely to rise to the top of the pay' pile.

----

Target: 1 (real disaster)
Text:Police Monitoring 200 Jihadis in Poland http://t.co/1wCOfmLUb9 via @freedomoutpost

----

Target: 0 (not real disaster)
Text:KS except every character is Shizune.
The world would explode.

----

Target: 0 (not real disaster)
Text:China's Stock Market Crash: Are There Gems In The Rubble? http://t.co/3PBFyJx0yA

----



### Split data into training and validation sets

In [12]:
from sklearn.model_selection import train_test_split


In [13]:
# Use train_test_split to split training data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1,
                                                                            random_state=42)

In [14]:
# check the lengths
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [17]:
# check the first 10 samples
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), array([0, 

## Converting text into numbers

Wheb dealing with a text problem, one of the first thing we have to do is convert text to numbers.

There are a few ways to do this, namely:  
* **Tokenization** - direct mapping of a token (a token could be a word or a character) to number (text vectorization).
* **Embedding** - create a matrix of feature vector for each token (the size of the feature vector can be defined and this embedding can be learned).

### Text vectorization (Tokenization)

In [18]:
train_sentences[:5]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
      dtype=object)

In [20]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# # Use the default text vectorization parameters
# text_vectorizer = TextVectorization(max_tokens=None, # how many words in the vocabulary (automatically add <OOV>)
#                                     standardize="lower_and_strip_punctuation",
#                                     split="whitespace",
#                                     ngrams=None, # creates groups of n words
#                                     output_mode="int",
#                                     output_sequence_length=None, # how long do you want your sequences to be?
#                                     pad_to_max_tokens=True)


In [21]:
train_sentences[0]

'@mogacola @zamtriossu i screamed after hitting tweet'

In [22]:
train_sentences[0].split()

['@mogacola', '@zamtriossu', 'i', 'screamed', 'after', 'hitting', 'tweet']

In [23]:
len(train_sentences[0].split())

7

In [25]:
# find the average number of tokens (words) in the training tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

Average number of tokens (words) in a tweet is 15.

In [27]:
# Setup text vectorization variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15  # max length our sequences will be (e.g how many words from a tweet our model will see)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)


In [29]:
# fit the text vectorizer to training data
text_vectorizer.adapt(train_sentences)

In [30]:
# create a sample sentence and tokenize it
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [32]:
# Choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\
        \n\n Vectorized version:")
text_vectorizer([random_sentence])

Original text:
 Spokane authorities say they're struggling to solve arson cases like today's on Hamilton. http://t.co/Qbs2k01WzK http://t.co/mvLZIYsGLL        

 Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[8206, 2188,  132,  723, 4517,    5, 4576,  612, 1702,   25, 1169,
          11, 3792,    1,    1]])>

In [33]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()  # get all of the unique words in the our training data
top_5_words = words_in_vocab[:5] # get the most common words 
bottom_5_words = words_in_vocab[-5:] # get the least common words
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f'5 most common words: {top_5_words}')
print(f"5 least common words: {bottom_5_words}")

Number of words in vocab: 10000
5 most common words: ['', '[UNK]', 'the', 'a', 'in']
5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


[UNK] - unknown token - stands for unknown (word out of the vocabulary)

### Creating an Embedding using an Embedding Layer

To make our embedding, we're going to use Tensorflow's embedding layer.

The parameters we care most about for our embedding layer:    
* `input_dim` = the size of our vocabulary
* ` output_dim` = the size of the output embedding vector, for example, a value of 100 would mean each token gets represented by a vector 100 long
* `input_length` = length of the sequences being passed to the embedding layer

In [34]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length, # set input shape
                             output_dim=128,
                             input_length=max_length)

embedding

In [37]:
# Get a random sentence from the training set
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\
        \n\nEmbedded version:")

# embed the random sentence (turn positive indexes into dense vectors of fixed size)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
 Watch how bad that fool get burned in coverage this year. Dat dude is all-pro practice squad material        

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-4.8269738e-02, -3.2349423e-02, -3.9045941e-02, ...,
         -2.1144975e-02,  3.0421380e-02, -2.3394752e-02],
        [-4.0663280e-02, -1.7847382e-02, -4.0422868e-02, ...,
          2.3357045e-02, -1.9212438e-02, -4.9503148e-02],
        [ 4.6793904e-02, -1.4097203e-02,  3.9659943e-02, ...,
          2.7140904e-02, -1.7818868e-02, -1.6926311e-02],
        ...,
        [ 1.5055206e-02,  4.1227713e-03, -6.1041005e-03, ...,
         -3.3744682e-02,  3.0510250e-02,  2.6273478e-02],
        [-4.0092468e-02, -1.6740583e-02,  2.8402101e-02, ...,
          3.4373272e-02, -1.8900288e-02, -6.9085583e-03],
        [ 2.0097271e-03, -3.0165685e-02,  9.0874732e-05, ...,
         -4.2057563e-02, -1.3209641e-02,  1.2419000e-03]]], dtype=float32)>

In [39]:
# check out a single token's embedding
sample_embed[0][0], sample_embed[0][0].shape, random_sentence.split()[0]

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-0.04826974, -0.03234942, -0.03904594, -0.03699374, -0.01734988,
        -0.00017799,  0.00637654, -0.02174883,  0.02768112,  0.0348699 ,
         0.02410794,  0.04600271,  0.01436564,  0.04202579,  0.02978617,
        -0.04971224, -0.04480026,  0.03728366, -0.02465379,  0.01527504,
         0.03347857, -0.04933841,  0.03299509, -0.03255733,  0.03123201,
        -0.02466154, -0.00877827,  0.04556871,  0.01155393,  0.04038162,
         0.00887102, -0.03249472,  0.00260364, -0.03622943,  0.00946524,
         0.04630331, -0.00757726, -0.01224137, -0.03646442,  0.02342888,
        -0.02339295, -0.02210646, -0.00582348,  0.04402863, -0.01607279,
        -0.02099642,  0.03932238, -0.03347111, -0.03150504, -0.04736254,
         0.02951889, -0.0249538 ,  0.00492309,  0.03616885, -0.01332701,
        -0.03731757,  0.02396596, -0.01766067,  0.02356834,  0.03339327,
        -0.01376615,  0.00285042, -0.03679176, -0.01322095, -0.01154441,
  